## Load in imports and data

Last working weekly: w2024_25

In [1]:
REPO = '/sdf/data/rubin/repo/main/'

from lsst.daf.butler import Butler
from pprint import pprint
import lsst.afw.image as afwImage
from lsst.skymap import Index2D
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

%matplotlib inline

butler = Butler(REPO)
registry = butler.registry

# define image display backend
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('firefly')

# NO_DATA investigation
import lsst.geom as geom

In [2]:
collection = 'u/mgorsuch/assemble_cell_coadd_patch_61_d0508' # updated patch 61 collection

In [3]:
coadd = butler.get('deepCoaddCell', 
                     collections=collection, 
                     instrument='HSC', 
                     skymap = 'hsc_rings_cells_v1', 
                     tract = 9813, 
                     patch=61,
                     band='i',)

In [4]:
stitch_coadd = coadd.stitch()

## Investigate NO_DATA

Find the cell that contains a large amount of NO_DATA masked pixels.
- The 2305, 715 coordinate is found using the Firefly display and hovering over the masked pixels. The readout option should be set to FITS Image Pixel. However, the coadd object does not have an origin at 0,0. This offset must be added, and is found using `getMin()` method.

In [5]:
print(coadd.grid.index(coadd.inner_bbox.getMin()+(geom.Extent2I(2305, 715))))

Index2D(x=15, y=4)


Alterate way to determine NO_MASK cell

In [6]:
print(coadd.grid.index(geom.Point2I(20105,15512)))

Index2D(x=15, y=4)


Find the input warps used in this cell:

In [7]:
cell_no_data = coadd.cells[Index2D(x=15, y=4)]
print(cell_no_data.inputs)

(ObservationIdentifiers(instrument='HSC', physical_filter='HSC-I', visit=19684, day_obs=20150121, detector=50),)


Using the output above, read in the relevant warp:

In [8]:
# get the above warp with no_data from the butler
warp2 = butler.get('deepCoadd_directWarp',
                   collections = collection,
                   instrument='HSC',
                   skymap = 'hsc_rings_cells_v1',
                   tract=9813,
                   patch=61,
                   visit = 19684)

#### Compare the warp, stitiched coadd, and cell images

Read in the relevant warp and the cell with NO_DATA masked pixels:

In [9]:
cell_test = coadd.cells[Index2D(x=15, y=4)]

In [10]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(warp2.getMaskedImage())
display2 = afwDisplay.Display(frame=2)
display2.mtv(coadd.cells[Index2D(x=15, y=4)].inner.asMaskedImage())

Read in the stitched coadd and the cell with NO_DATA masked pixels:

In [11]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(warp2.variance)
display2 = afwDisplay.Display(frame=2)
display2.mtv(coadd.cells[Index2D(x=0, y=0)].inner.variance)

In [12]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(stitch_coadd.asMaskedImage())
display2 = afwDisplay.Display(frame=2)
display2.mtv(coadd.cells[Index2D(x=15, y=4)].inner.asMaskedImage())

### Investigate NO_DATA in normal warping/coaddition process (none cell-based)
**TO-DO**: need to rerun with updated dataset types

Read in the relevant warp and coadd